In [1]:
import sys
import os
from pathlib import Path
import torch
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
from utils import *
from configuration import *
import numpy as np

if not os.path.exists('./evaluation1/'):
    os.makedirs('./evaluation1/')

In [2]:
args = parser.parse_args([])
args.DEVICE = 'cpu'
args.task = 'temporal'
args.metric = 'temporal_acc'
args.SoftEva = True
args = FormulateArgs(args)

## Results on non-augmented test set

In [3]:
args.augment = False
args.augment

False

In [4]:
results = torch.zeros([15,10,6])

for ds in [9, 10, 11, 12, 13, 14]:
    args.DATASET = ds
    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
    test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')
    print(datainfo)
    for seed in range(10):
        args.SEED = seed

        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        
        model_exist = os.path.isfile(f'./models/{modelname}')
        aug_model_exist = os.path.isfile(f'../AugLearnableFilters/models/{modelname}')
        # baseline_model_exist = os.path.isfile(f'../LastBaseline/LearnableFilters/models/{modelname}')
        
        # print(model_exist and aug_model_exist and baseline_model_exist)
        print(model_exist and aug_model_exist)
        
        
        if model_exist and aug_model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            aug_model = torch.load(f'../AugLearnableFilters/models/{modelname}', map_location=args.DEVICE)
            # baseline_model = torch.load(f'../LastBaseline/LearnableFilters/models/{modelname}', map_location=baseline_args.DEVICE)
            
            model.UpdateArgs(args)
            aug_model.UpdateArgs(args)
            # baseline_model.UpdateArgs(baseline_args)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)
            # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            acc_valid = evaluator(model, X_valid, y_valid)
            acc_test   = evaluator(model, X_test,  y_test)
            
            aug_acc_valid = evaluator(aug_model, X_valid, y_valid)
            aug_acc_test   = evaluator(aug_model, X_test,  y_test)
            
            # baseline_acc_valid = baseline_evaluator(baseline_model, X_valid, y_valid)
            # baseline_acc_test   = baseline_evaluator(baseline_model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
            
            results[ds,seed,2] = aug_acc_valid
            results[ds,seed,3] = aug_acc_test
            
            # results[ds,seed,4] = baseline_acc_valid
            # results[ds,seed,5] = baseline_acc_test
        else:
            results[ds,seed,:] = float('nan')

{'dataname': 'powercons', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 216, 'N_valid': 72, 'N_test': 72}
False
False
False
False
False
False
False
False
False
False
{'dataname': 'proximalphalanxoutlinecorrect', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
False
False
False
False
False
False
False
False
False
False
{'dataname': 'selfregulationscp2', 'N_feature': 7, 'N_class': 2, 'N_time': 64, 'N_train': 228, 'N_valid': 76, 'N_test': 76}
False
False
False
False
False
False
False
False
False
False
{'dataname': 'slope', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 300, 'N_valid': 100, 'N_test': 100}
False
False
False
False
False
False
False
False
False
False
{'dataname': 'smoothsubspace', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 180, 'N_valid': 60, 'N_test': 60}
False
False
False
False
False
False
False
False
False
False
{'dataname': 'symbols', 'N_feature': 1, 'N_class': 6, 'N_time': 64, 'N_train': 300, 'N_vali

In [43]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,1], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation1/non_var_test_top_{N_selected_seeds}_var_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')

In [44]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,3], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation1/non_var_test_top_{N_selected_seeds}_aug_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')

## Results on augmented test set

In [4]:
args.augment = True
args.augment

True

In [6]:
results = torch.zeros([15,10,6])
for ds in range(9):
    args.DATASET = ds
    valid_loader, datainfo = GetDataLoader(args, 'valid', path='../dataset/')
    test_loader , datainfo = GetDataLoader(args, 'test',  path='../dataset/')  
    print(datainfo)
    for seed in range(10):
        args.SEED = seed


        modelname = f"pLF_data_{ds:02d}_{datainfo['dataname']}_seed_{seed:02d}.model"
        
        model_exist = os.path.isfile(f'./models/{modelname}')
        aug_model_exist = os.path.isfile(f'../AugLearnableFilters/models/{modelname}')
        # baseline_model_exist = os.path.isfile(f'../LastBaseline/LearnableFilters/models/{modelname}')
        
        # print(model_exist and aug_model_exist and baseline_model_exist)
        print(model_exist and aug_model_exist)
        
        
        if model_exist and aug_model_exist:  
                          
            model = torch.load(f'./models/{modelname}', map_location=args.DEVICE)
            aug_model = torch.load(f'../AugLearnableFilters/models/{modelname}', map_location=args.DEVICE)
            # baseline_model = torch.load(f'../LastBaseline/LearnableFilters/models/{modelname}', map_location=baseline_args.DEVICE)
            
            model.UpdateArgs(args)
            aug_model.UpdateArgs(args)
            # baseline_model.UpdateArgs(baseline_args)
            
            SetSeed(args.SEED)

            evaluator = Evaluator(args).to(args.DEVICE)
            # baseline_evaluator = baselineEvaluator(baseline_args).to(baseline_args.DEVICE)

            for x,y in valid_loader:
                X_valid, y_valid = x.to(args.DEVICE), y.to(args.DEVICE)
            for x,y in test_loader:
                X_test, y_test = x.to(args.DEVICE), y.to(args.DEVICE)

            acc_valid = evaluator(model, X_valid, y_valid)
            acc_test   = evaluator(model, X_test,  y_test)
            
            aug_acc_valid = evaluator(aug_model, X_valid, y_valid)
            aug_acc_test   = evaluator(aug_model, X_test,  y_test)
            
            # baseline_acc_valid = baseline_evaluator(baseline_model, X_valid, y_valid)
            # baseline_acc_test   = baseline_evaluator(baseline_model, X_test,  y_test)

            results[ds,seed,0] = acc_valid
            results[ds,seed,1] = acc_test
            
            results[ds,seed,2] = aug_acc_valid
            results[ds,seed,3] = aug_acc_test
            
            # results[ds,seed,4] = baseline_acc_valid
            # results[ds,seed,5] = baseline_acc_test
        else:
            results[ds,seed,:] = float('nan')

{'dataname': 'cbf', 'N_feature': 1, 'N_class': 3, 'N_time': 64, 'N_train': 600, 'N_valid': 200, 'N_test': 200}
True
True
True
True
True
True
True
True
True
True
{'dataname': 'distalphalanxtw', 'N_feature': 1, 'N_class': 6, 'N_time': 64, 'N_train': 600, 'N_valid': 200, 'N_test': 200}
False
True
True
False
True
True
True
True
False
True
{'dataname': 'freezerregulartrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 600, 'N_valid': 200, 'N_test': 200}
True
True
True
True
True
True
True
True
True
True
{'dataname': 'freezersmalltrain', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 600, 'N_valid': 200, 'N_test': 200}
True
True
True
True
True
True
True
True
True
True
{'dataname': 'gunpointagespan', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 540, 'N_valid': 180, 'N_test': 182}
True
True
True
True
True
True
True
True
True
True
{'dataname': 'gunpointmaleversusfemale', 'N_feature': 1, 'N_class': 2, 'N_time': 64, 'N_train': 540, 'N_valid': 180, 'N_test': 182}
True
T

In [37]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,1], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation1/var_test_top_{N_selected_seeds}_var_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')

In [38]:
N_selected_seeds = 3
re_temp = torch.nan_to_num(results, nan=-10000.)
values, indices = torch.topk(re_temp[:,:,3], k=N_selected_seeds, dim=1)
mean_selected = torch.mean(torch.asarray(values), dim=1)
var_selected = torch.std(torch.asarray(values), dim=1)
selected_results = torch.cat([mean_selected.unsqueeze(-1), var_selected.unsqueeze(-1)], dim=1)

np.savetxt(f"./evaluation1/var_test_top_{N_selected_seeds}_aug_LPF_acc.txt", selected_results.numpy(), delimiter="\t", fmt='%.5f')